# MSI Project 2 –  Water chemical potential
**Mariona Torrens Fontanals**
*****

## Introduction

The following python module, given a set of molecular dynamics trajectories, computes the volumetric map of the chemical potential for water.  
To do that, we compute the average density of water (water occupancy) around the protein by counting the number of oxygen water atoms found at each cell of a tridimensional cubic grid which has the protein at the center. Then, we apply the Boltzmann equation to obtain the chemical potential from the occupancy.   

In order to properly execute this module, we need to have all the simulation trajectories folders stored in a directory called *data*, placed at the working directory.

The code used to achieve that is detailed above:

## Details of the module

The code is organized in several functions which are called by a main function, named `volumetric_map`.  
First of all, the main function creates a list of simulations (`simlist` method), which allows us to iterate through them. Therefore, the main part of the code is inside a `for` loop, so that it is applied for each simulation trajectory.

The `prepare_traject` function loads the pdb file associated to a given simulation *s* and the trajectory file. Also, the `wrap` method is used to enclose the coordinates of the Molecule object into a continuous cubic space, so that the water molecules don't escape away from the protein, which would make the calculation of the water occupancy impossible.  

The Molecule of each simulation needs to be aligned the same way as the others. For this reason, for the first simulation loaded we save a copy of the aligned Molecule (we name it *ref_traj*) and use it as a reference on which to align the Molecule of the following simulations.

The Molecule is centered at [0,0,0] to make sure that we start at the same position for all the simulations. Afterwards, we filter out the protein so that we keep only the oxygen atoms of the water molecules (*traj_wat*), which will later serve us as an approximation of where the water molecules are located.

In [5]:
from htmd import *
from htmd.molecule.util import maxDistance
from htmd.molecule.util import writeVoxels
import time

In [6]:
def prepare_traject(s,ref_traj=None):
    traj = (Molecule(s.molfile))
    traj.read(s.trajectory)
    traj.wrap('protein') #we want the prot to be the center of the wrapping box
    if ref_traj is None:
        traj.align('protein')
        ref_traj=traj.copy()
    else:
        traj.align('protein',ref_traj)
    coo = traj.get('coords')
    c = np.mean(coo, axis=0)
    traj.moveBy(-c)
    traj_wat= traj.copy()
    traj_wat.filter("water and name OH2")    
    return(traj,traj_wat,ref_traj)

We need a reference value which allows us to determine the dimensions of the grid. In order to obtain it, when we load the first simulation, the function `define_extremes` applies `maxDistance`, which calculates the maximum distance from the center of the Molecule to the furthest atom. With this value (*D*), we can calculate the starting and ending point of the grid at each dimension (x,y,z). The grid will be placed at positive coordinates, so its values of x, y and z will go from 0 (*min_d*) to two times the integer of *D* rounded up (*max_d*).

For all the simulations, we will use the grid dimensions calculated for the first one to be loaded, so we only need to execute `define_extremes` for this one. All the necessary information is saved so that we can use it for all the others.

In [7]:
def define_extremes(traj_wat):
    D = maxDistance(traj_wat)
    min_d = 0
    max_d = 2*int(D+1)    
    return(min_d,max_d)

With the purpose to obtain the tridimensional grid, we start by creating a dictionary (*w_count*) in which each key corresponds to a tuple (x,y,z), where x, y and z is an integer from 0 to *max_d*. The tuple (x,y,z) represents the grid cell that includes all the oxygen atoms of water with coordinates such that:
* x is within the interval (x,x+1)
* y is within the interval (y,y+1) 
* z is within the interval (z,z+1)

Therefore, our grid has a resolution of 1 Angstrom for each dimension.  
The value associated to each key, for the moment, is set to 0, but it will eventually be the density of oxygen atoms at the corresponding cell.

The mentioned dictionary is created by the function `create_count_dict`.

In [8]:
def create_count_dict(min_d,max_d):
    w_count={}
    for x in range(min_d,max_d+1):
        for y in range(min_d,max_d+1):
            for z in range(min_d,max_d+1):
                w_count[(x,y,z)]=0
    return(w_count)

`calculate_occupancy` iterates through the coordinates of all the oxygen atoms at *traj_wat*, considering all its frames. For each atom, sums 1 to the dictionary value of the corresponding grid cell. Also, in order to smoothen the isosurface that will result from all this, we add 0.5 to the grid cells surrounding the one where the atom is found. This makes the program slower, but with better results.

After that, we transform the dictionary to a matrix (*w_count_list*) of dimension *n.n.n*, where *n* is the number of grid cells along each axis. Each matrix cell contains a value of the number of waters in a grid cell. Of course, in the matrix, the values of water counts per grid cell are ordered by its spatial coordinates (thus, by the dictionary key).

The values of the matrix are divided:
* by the number of frames in the simulation, because we are interested in the average water occupancy of all frames
* and by number of oxygen atoms, so that we obtain the water density.

Therefore, we now have a matrix of the average water occupancy along the tridimensional space, for a given simulation (*avg_wat*).

In [9]:
def calculate_occupancy(traj_wat,w_count,min_d,max_d):
    print("Calculating water occupancy....")
    for atom_c in traj_wat.coords:
        for i in range(traj_wat.numFrames):
            x=int(atom_c[0,i])
            y=int(atom_c[1,i])
            z=int(atom_c[2,i])
            if min_d < x < max_d and min_d < y < max_d and min_d < z < max_d:
                w_count[(x,y,z)]+=1
                w_count[(x-1,y,z)]+=0.5
                w_count[(x+1,y,z)]+=0.5
                w_count[(x,y-1,z)]+=0.5
                w_count[(x,y+1,z)]+=0.5
                w_count[(x,y,z-1)]+=0.5
                w_count[(x,y,z+1)]+=0.5   
    w_count_list= np.array([w_count[key] for key in sorted(w_count)]).reshape(max_d+1,max_d+1,max_d+1)   
    avg_wat = w_count_list / (traj_wat.numFrames*traj_wat.numAtoms) 
    print("Done.")    
    return(avg_wat)

In the main function (`volumetric_map`), the *avg_wat* matrices of all the simulations are added, which gives a matrix of the summatory of water occupancies of all of them (*occup_all*). This matrix is divided by the total number of simulations, in order to obtain a matrix of average water occupancies of all the simulations.   
Afterwards, we sum to the matrix a very small number, such as 1e-50. We do this to avoid having zero values in it, which would be a problem at the following step of the program, when we have to convert to logarithm.

At this point, the function `chemical_pot` is executed. It applies the Boltzmann equation, and in consequence we obtain a matrix of chemical potentials (*pot*) from the matrix of water occupancies.

In [10]:
def chemical_pot(occupancy):
    kB=0.001987191 
    T=298 
    pot = -kB*T*np.log(occupancy)        
    return(pot)

The main function (`volumetric_map`) now applies `writeVoxels` in order to create a cube file from the matrix *pot*. The cube file, named *cubefile.cube* is saved at the working directory.

This file can be visualized with VMD on top of the PDB of the molecule as an isosurface. For this reason, the function `view_vmd` opens with VMD the trajectory of one of the simulations (the last one, concretely) so that we can load, manually, the cubefile on it.

In [11]:
def view_vmd(traj, max_d):    
    traj.moveBy([max_d/2,max_d/2,max_d/2])
    htmd.config(viewer='vmd')
    traj.reps.remove()   
    traj.reps.add(sel='protein', style='NewCartoon', color='2')
    traj.view()
    time.sleep(6000)

`volumetric_map` is the main function of the module, which, when executed, calls all the other ones. Therefore, in order to run the program we just need to execute this function.

In [12]:
def volumetric_map():
    sims = simlist(glob('data/*/'), glob('data/*/structure.pdb'), glob('input/*/'))
    sim_num=len(sims)
    for s in sims:
        print("\nSimulation", s.simid +1, "of", sim_num)
        if s.simid == 0:
            (traj,traj_wat,ref_traj)=prepare_traject(s)
            (min_d,max_d)=define_extremes(traj_wat)
        else:
            (traj,traj_wat,ref_traj)=prepare_traject(s,ref_traj)
        traj_wat.moveBy([max_d/2,max_d/2,max_d/2])
        w_count=create_count_dict(min_d,max_d)
        avg_wat = calculate_occupancy(traj_wat,w_count,min_d,max_d)
        try:
            occup_all += avg_wat
        except NameError:
            occup_all = avg_wat
    occup_all=occup_all/sim_num
    occup_all += 1e-50
    pot=chemical_pot(occup_all)
    writeVoxels(pot,"cubefile.cube",np.array([min_d,min_d,min_d]),np.array([max_d,max_d,max_d]),np.array([1,1,1]))
    print("\nVolumetric file created.\n")
    view_vmd(traj,max_d)

In [ ]:
volumetric_map()

## Results

In the image below you can see the resulting isosurface (white), represented on top of the protein (red):


![](isosurface.png)